## Notebook 1

This  note should be run twice, first time without any tranformations and second time with the transformations. 
The user can find the code in the 
This notebook applies the transformations/ data preprocessing on the datasets and calculates the multi CVI rank correlation again and concatenates the new informations in the Knowledge space. The knowledge space/ meta data includes the extracted meta features and applied transformations, combination of the multi CVIs and the correlation score. 

### Initialization
The libraries required for the experiment is in the following code cell

In [ ]:
#Required libraries. 
#install deap if not installed 

#pip install deap
import itertools
from scipy.spatial import distance
import numpy as np
import math
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.datasets import load_iris
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering, Birch
import warnings
from sklearn.metrics import normalized_mutual_info_score, davies_bouldin_score
from sklearn.cluster import SpectralClustering

from metafeatures import Meta
from deap import base
from deap import creator, tools
from scipy.stats import spearmanr
from cvi import Validation
from sklearn.preprocessing import OrdinalEncoder
import glob
from scipy.io import arff
import pandas as pd
from cvi import Validation
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from itertools import combinations

### Defining the multi CVI functions
The following cell holds the definition of the the few Multi CVIs that were either used for the experiments. Some of them were not used for the final evaluation.

In [ ]:
#C-index CVI
def c_index(data, class_label):
    """
        The C-Index, a measure of dispersionn
    """
    sw = 0
    nw = 0
    numCluster = max(class_label) + 1
    data_matrix = np.asmatrix(data).astype(np.float)
    
    # iterate through all the clusters
    for i in range(numCluster):
        indices = [t for t, x in enumerate(class_label) if x == i]
        clusterMember = data_matrix[indices, :]
        # compute distance of every pair of points
        list_clusterDis = distance.pdist(clusterMember)
        sw = sw + sum(list_clusterDis)
        nw = nw + len(list_clusterDis)
    # compute the pairwise distance of the whole dataset
    list_dataDis = distance.pdist(data_matrix)
    # compute smin
    sortedList = sorted(list_dataDis)
    smin = sum(sortedList[0:nw])
    # compute smax
    sortedList = sorted(list_dataDis, reverse=True)
    smax = sum(sortedList[0:nw])
    
    # compute the score
    return (sw - smin) / (smax - smin)


#i-index CVI  
def i_index(data, class_label):
      """
        The I index, a measure of compactness.
      """
      
      normClusterSum = 0
      normDatasetSum = 0
      list_centers = []

      data_matrix = np.asmatrix(data).astype(np.float)
      # compute the number of clusters and attribute
      attributes = len(data_matrix[0])
      numCluster = max(class_label) + 1
      # compute the center of the dataset
      dataCenter = np.mean(data_matrix, 0)
      # iterate through all the clusters
      for i in range(numCluster):
          indices = [t for t, x in enumerate(class_label) if x == i]
          clusterMember = data_matrix[indices, :]
          # compute the center of the cluster
          clusterCenter = np.mean(clusterMember, 0)
          list_centers.append(np.asarray(clusterCenter))
          # compute the norm for every member in the cluster with cluster center and dataset center
          for member in clusterMember:
              normClusterSum += distance.euclidean(member, clusterCenter)
              normDatasetSum += distance.euclidean(member, dataCenter)
      # compute the max distance between cluster centers
      list_centers = np.concatenate(list_centers, axis=0)
      maxCenterDis = max(distance.pdist(list_centers))
      
      # compute the fitness      
      return math.pow(((normDatasetSum * maxCenterDis) / (normClusterSum * numCluster)), attributes)
    

#Banfel Rafery CVI
def banfeld_raferty(data, class_label):
    """ Banfeld-Raferty index is the weighted sum of the logarithms
         of the traces of the variance-covariance matrix of each cluster
         
        Weighted sum of the logarithms of the traces of the variance-covariance matrix of each cluster
        
        OBJECTIVE: MIN
    """

    sum_total = 0
    num_cluster = max(class_label) + 1
    data_matrix = np.asmatrix(data).astype(np.float)
    
    # iterate through all the clusters
    for i in range(num_cluster):
        sum_dis = 0
        indices = [t for t, x in enumerate(class_label) if x == i]
        cluster_member = data_matrix[indices, :]

        # compute the center of the cluster
        cluster_center = np.mean(cluster_member, 0)

        # iterate through all the members
        for member in cluster_member:
            sum_dis += distance.euclidean(member, cluster_center) ** 2

        op = sum_dis / len(indices)
        if op <= 0:
            # Cannot calculate Banfeld_Raferty, due to an undefined value
            continue
        else:
            sum_total += len(indices) * math.log(sum_dis / len(indices))

    return sum_total

### Setting configuration
The following cell includes setting the hyperparameter range including the clustering algorithm. 
For the sake of the experiment, we used three clustering algorithm, KMeans, Agglomerative, and Birch. \\

The each algorithm was applied sperately at one time. User can comment/ uncomment to use parts of the code in the following cells. 

In [ ]:

"""
    Generates different cluster configurations from the KMeans, Agglomerated, Birch Clustering algorithm, 
    modifying hyper-parameters: n_clusters, 
"""

#keeping the configuration in a list
configs = list()
"""
#Agglo
for i in np.arange(2, 20, 1):
    configs.append(AgglomerativeClustering( n_clusters=i))

#for j in np.arange(0.5, 1.5, 0.5):
#    configs.append(AgglomerativeClustering(distance_threshold=j, n_clusters=None))
   
"""

##K-Means
for i in list(range(100, 101,1)):
    #configs.append(AgglomerativeClustering(n_clusters=i))

    for j in list(range(2, 20,1)):
        configs.append(KMeans(n_clusters=j, max_iter=i)) #user can use other clustering algorithm for each phase here

"""
#Birch
for i in np.arange(2, 20, 1):
    configs.append(Birch( n_clusters=i))
  
#Spectral####not performed
for i in np.arange(2, 20, 1):
    configs.append(SpectralClustering( n_clusters=i))
"""
#we need the length of the configuration for ranking later
len(configs)

### Transforming, clustering, and calculating multi CVI

In [ ]:
"""
    Multi CVI uses DEAP, an Evolutionary Algorithm implementation we will use in later tasks
    Goal here is to sort cluster configurations with multi-objective functions, using the Non-dominated sorting algorithm II (NSGA-2)
    More about  NSGA-2 here: https://ieeexplore.ieee.org/document/6600851

    Parameter Definition REQUIRED
"""



#initializing the preprocessing methods
kbin = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')
norm_l1=Normalizer(norm='l1')
norm_l2=Normalizer(norm='l2')
std_zscore=StandardScaler()
scaling_minmax=MinMaxScaler()
#replacing_mean=SimpleImputer(missing_values=np.nan, strategy='mean')
#replacing_median=SimpleImputer(missing_values=np.nan, strategy='median')

"""
    This the part where the user needs to disable the transformations to prevent transforming the data first 
    time, and then cluster and calculate the multi CVI correlation score ones. 
    
    Then for second time, the user needs to enable the transformations, to transform the data and cluster and
    calculate the multi CVI correlation score again. 
    
    Each time I saved the dataframe and later cocatenated them in other notebook for analysis. 
    
    For transofrmaiton, keeping the preprocessing methods in a list so that we can use a for loop later tto iterate over them
    Later i manually used mean and median manaully as the packageeee was giving some error. 
"""

transformations = [kbin, norm_l1, norm_l2, scaling_minmax, std_zscore]#, replacing_mean, replacing_median]

"""
    This list is for creating all the combinations of transformations in a pair, i.e. [kbin, norm_l1], [norm_l1, norm_l2]
    we apply this to create an additional knowledge applying two transformations at once on each dataset to check
    if applying two preprocessing method improves/ degrades the result
"""
tr_list=list(combinations([0,1,2,3, 4],2))


"""
    Initializing the knowledge base dataframe where we will store the information, meta features of the datasets
"""

df_meta=pd.DataFrame(columns=['name', 'instances', 'attributtes', 'classes', 'cat_att', 'cont_att', 'mean', 'sd', 'var', 'kurtosis', 'skewness', 'MD6', 'MD7', 'MD8', 'MD9',
       'MD10', 'MD11', 'MD12', 'MD13', 'MD14', 'MD15', 'MD16', 'MD17', 'MD18',
       'MD19', 'dataset', 'best_cluster_setting', 'transformation','multi_cvi', 'multi_cvi_correlation_score'])

#Path to the datasets
files=glob.glob(r"/Users/hasan.tanvir/Documents/Thesis/Data/clustering-benchmark-master/src/main/resources/datasets/artificial/*.arff")
print(len(files))

for t in transformations:

    

    for file in files:
        print(file)
        
        #read the dataset
        data = arff.loadarff(file)
        df = pd.DataFrame(data[0])
        df.columns = [x.lower() for x in df.columns]
        cols=df.columns
        num_cols = df._get_numeric_data().columns
        df=df.dropna() 
        
        #check if the dataset has column class & some preprocessing
        if df.shape[1]>2:
            df['class'] = df['class'].str.decode('utf-8') 
            df['class']=df['class'].replace('noise','3',regex=True)

            try:# int(df['class'][0]): #for classes where class labels are integer
                df['class']=df['class'].astype(str).astype(int)
                
                #getting the file name
                new_file = file.split('/')[-1].split('.')[0]
                #print(new_file)

            except: #for classes where the class labels are not integer
                df_cols=df.columns
                encoder = OrdinalEncoder()
                # transform data
                #result = encoder.fit_transform(df)
                df=pd.DataFrame(result, columns=df_cols)

                new_file = file.split('/')[-1].split('.')[0]
                print(new_file)
#"""
#    Using the file name to extract metafeature, for extracting meta features, I used the csmartml package
#    and the package requires the files to be read in CSV. So I converted the files in CSV beforehand so that we 
#    can perform this step
#"""
        df1=Meta("csmartml/datasets/"+new_file+".csv").extract_metafeatures(meta_type="distance")

        
        meta_dict=df1.to_dict('dict')   
#"""
#    This is the part where we apply mean and median manually as the package give some kind of erros. 
#    Since the dataset does not have empty/ null/ NA values, so far I guess, I manually, randomly remove 20%
#    of the data and then impute those with mean and media.     
#"""

        #for col in df.columns:
        #    df.loc[df.sample(frac=0.2).index, col] = np.nan 

        #df.apply(lambda x: x.fillna(x.mean(), inplace=True),axis=0)

        X=df[df.columns.difference(['class'])].values
        y=df['class'].values


        #df=df.dropna() 
        #print(X)
        X=t.fit_transform(df[df.columns.difference(['class'])].values)

#"""
#    The following section directly comes from SmartML homework, thatI received from Hudson. This is where
#    the multi CVI are ranked using NSGA-2 and the corraltion scores are calculated. I tried to use/ reproduce
#    it dunamically, but that seemed to be too much of work hence, I refrained. 
    
#    The only change I made is added one more CVI to make it a combination of 3 and added the optimization 
#    objective in the objecitve function to define the objective of the new CVI. I believe this part won't be 
#    needed for the cSmartML project if you want to integrate preprocessing part in the tool later. 
#"""
        def fitness_function(individual):
            clustering = individual[0].fit(X)
            #print(individual[0])
            labels = clustering.labels_

            #if len(set(labels))==1:
                #return 0
            #print('clus',set(labels))
            #print(labels)
            #print('X', len(X))
            return Validation(X, df.values,labels).s_dbw(), davies_bouldin_score(X, labels),banfeld_raferty(X, labels)

        def initIndividual(icls, content):
            return icls([content])

        def initPopulation(pcls, ind_init, poplist):
            return pcls(ind_init(c) for c in poplist)

        # Think of creator as an abstraction for classes that can be modified for evolutionary algorithms.
        # Eg. in this case is the base Fitness class
        # weights = tuple defining objective of CVI, max: 1 or min: -1
        # weight should be fixed according to the objective function of the CVI, respectively
        creator.create("FitnessMulti", base.Fitness, weights=(-1.0, -1.0, -1.0)) 
        creator.create("Individual", list, fitness=creator.FitnessMulti)

        # Toolbox contains specific functions for evolutionary operations
        # Read more: https://deap.readthedocs.io/en/master/api/base.html?highlight=toolbox#toolbox
        toolbox = base.Toolbox()
        toolbox.register("individual", initIndividual, creator.Individual)
        toolbox.register("population", initPopulation, list, toolbox.individual, configs)
        toolbox.register("evaluate", fitness_function)
        toolbox.register("select", tools.selNSGA2)

#        """
#            Rank configuration by multiple CVIs
#        """
        pop = toolbox.population()

        pop_dict = {} # Numbering configurations so it's easy to compare their ranks
        for i, j in enumerate(pop):
            pop_dict[i] = j

        invalid_ind = [ind for ind in pop if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        
        #Fix the value for toolbox.select(pop, N), the N should be the numbe rof configurations we have from 
        #the previous cell, could have been dynamic, but I missed :) 
        
        nsga_sorted = toolbox.select(pop,18)
        multi_cvi_rank = [list(pop_dict.values()).index(i) for i in nsga_sorted]
        multi_cvi_rank
        #print(pop_dict[multi_cvi_rank[0]])


        #CVI part



        cvi_1_dict = {}
        cvi_2_dict = {}
        cvi_3_dict = {}

        for k, v in pop_dict.items():
            clustering = v[0].fit(X)

            cvi_1_dict[k] = Validation(X, X, clustering.labels_).s_dbw()
            cvi_2_dict[k] = davies_bouldin_score(X, clustering.labels_)
            cvi_3_dict[k] = banfeld_raferty(X, clustering.labels_)

        cvi_1_dict = dict(sorted(cvi_1_dict.items(), key=lambda item: item[1], reverse=False))
        cvi_2_dict = dict(sorted(cvi_2_dict.items(), key=lambda item: item[1], reverse=False))
        cvi_3_dict = dict(sorted(cvi_3_dict.items(), key=lambda item: item[1], reverse=False))

        cvi_1_rank = list(cvi_1_dict.keys())
        cvi_2_rank = list(cvi_2_dict.keys())
        cvi_3_rank = list(cvi_3_dict.keys())


        # Rank configurations by external CVI (NMI) 

        ground_truth_dict = {}
        for k, v in pop_dict.items():
            clustering = v[0].fit(X)
            ground_truth_dict[k] = normalized_mutual_info_score(y, clustering.labels_)
            #print(ground_truth_dict)

        ground_truth_dict = dict(sorted(ground_truth_dict.items(), key=lambda item: item[1], reverse=True))
        gt_cvi_rank = list(ground_truth_dict.keys())
        #print(gt_cvi_rank)
        cvi_1_correlation = spearmanr(cvi_1_rank, gt_cvi_rank)
        cvi_2_correlation = spearmanr(cvi_2_rank, gt_cvi_rank)
        cvi_3_correlation = spearmanr(cvi_3_rank, gt_cvi_rank)
        multi_correlation = spearmanr(multi_cvi_rank, gt_cvi_rank)

        #print("Correlation with SDbw: ", cvi_1_correlation.correlation)
        #print("Correlation with Davies Bouldin Score: ", cvi_2_correlation.correlation)
        #print("Correlation with Banfeld Raferty: ", cvi_3_correlation.correlation)
        #print("Original NMI", gt_cvi_rank)
        #print("Correlation with multi CVI: ", multi_correlation.correlation)
        #print(meta_dict)
        
        #Finding out the categorical columns
        cat_cols=df.iloc[:, :-1].columns
        cat_num_cols = df.iloc[:, :-1]._get_numeric_data().columns

         #print(gt_cvi_rank)
        
        #pushing everythign to the dataframe to create the knowledge base
        df_meta=df_meta.append({'name': new_file, 
                             'instances':df.shape[0], 
                             'attributtes':df.shape[1]-1, 
                             'classes':df['class'].nunique(), 

                             'cat_att':len(list(set(cat_cols) - set(cat_num_cols))),
                             'cont_att':len(num_cols),
                             #'multi_CVI': key,
                             #'score':value, 
                            'mean': meta_dict.get('mean')[0],
                            'sd': meta_dict.get('sd')[0],
                            'var': meta_dict.get('var')[0],
                            'kurtosis': meta_dict.get('kurtosis')[0],
                            'skewness': meta_dict.get('skewness')[0],
                            'MD6': meta_dict.get('MD6')[0],
                            'MD7': meta_dict.get('MD7')[0],
                                'MD8': meta_dict.get('MD8')[0],
                                'MD9': meta_dict.get('MD9')[0],
                                'MD10': meta_dict.get('MD10')[0],
                                'MD11': meta_dict.get('MD11')[0],
                                'MD12': meta_dict.get('MD12')[0],
                                'MD13': meta_dict.get('MD13')[0],
                                'MD14': meta_dict.get('MD14')[0],
                                'MD15': meta_dict.get('MD15')[0],
                                'MD16': meta_dict.get('MD16')[0],
                                'MD17': meta_dict.get('MD17')[0],
                                'MD18': meta_dict.get('MD18')[0],
                                'MD19': meta_dict.get('MD19')[0],
                                'best_cluster_setting':pop_dict[multi_cvi_rank[0]],
                                'transformation':str("mean + "+str(t)),
                                'multi_cvi':'banfeld_raferty, davies_bouldin_score, SDbw',


                                'multi_cvi_correlation_score': multi_correlation.correlation}, ignore_index=True)




In [ ]:
#save the dataframe
df_meta.to_csv("transformation_birch_mean.csv")
